# Accident Severity Prediction Notebook

In [1]:
#Download intial packages

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [4]:
#download dataset

!wget -O Data-Collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

--2020-09-02 18:30:04--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘Data-Collisions.csv’

100%[======================================>] 73,917,638  44.0MB/s   in 1.6s   

2020-09-02 18:30:06 (44.0 MB/s) - ‘Data-Collisions.csv’ saved [73917638/73917638]



In [5]:
df_collisions = pd.read_csv('Data-Collisions.csv')
df_collisions.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [6]:
#create subset with just the desired features
df_collide = df_collisions[['SEVERITYCODE','LIGHTCOND','ROADCOND','WEATHER']]

In [7]:
df_collide.head()

,SEVERITYCODE,LIGHTCOND,ROADCOND,WEATHER
0,2,Daylight,Wet,Overcast
1,1,Dark - Street Lights On,Wet,Raining
2,1,Daylight,Dry,Overcast
3,1,Daylight,Dry,Clear
4,2,Daylight,Wet,Raining


In [8]:
df_collide['WEATHER'].value_counts()

Clear                       111135
Raining                      33145
Overcast                     27714
Unknown                      15091
Snowing                        907
Other                          832
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64

In [9]:
#Make a dictionary that replaces the values of the features with numerals
cleanup_nums = {"LIGHTCOND": {"Daylight": 0, "Dark - Street Lights On": 3, "Unknown": 8, "Dusk": 1, "Dawn": 2, "Dark - No Street Lights": 4,
                                 "Dark - Street Lights Off": 5, "Other": 7, "Dark - Unknown Lighting": 6},
                "ROADCOND": {"Dry": 0, "Wet": 1, "Unknown": 8, "Ice": 2, "Snow/Slush": 3, "Other": 7, "Standing Water": 6, "Sand/Mud/Dirt": 5,
                            "Oil": 4},
               "WEATHER": {"Clear": 0, "Raining": 3, "Overcast": 2, "Unknown": 10, "Snowing": 4, "Other": 9, "Fog/Smog/Smoke": 6, 
                          "Sleet/Hail/Freezing Rain": 5, "Blowing Sand/Dirt": 7, "Severe Crosswind": 8, "Partly Cloudy": 1}}

In [10]:
df_collide.replace(cleanup_nums, inplace=True)
df_collide

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/generic.py:6517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


,SEVERITYCODE,LIGHTCOND,ROADCOND,WEATHER
0,2,0.0,1.0,2.0
1,1,3.0,1.0,3.0
2,1,0.0,0.0,2.0
3,1,0.0,0.0,0.0
4,2,0.0,1.0,3.0
5,1,0.0,0.0,0.0
6,1,0.0,1.0,3.0
7,2,0.0,0.0,0.0
8,1,0.0,0.0,0.0
9,2,0.0,0.0,0.0


In [11]:
df_collide.head()

,SEVERITYCODE,LIGHTCOND,ROADCOND,WEATHER
0,2,0.0,1.0,2.0
1,1,3.0,1.0,3.0
2,1,0.0,0.0,2.0
3,1,0.0,0.0,0.0
4,2,0.0,1.0,3.0


In [12]:
#Eliminate rows with missing values
df_collide=df_collide.dropna()
df_collide.shape

(189337, 4)

In [13]:
df_collide.dtypes

SEVERITYCODE      int64
LIGHTCOND       float64
ROADCOND        float64
WEATHER         float64
dtype: object

In [14]:
#Convert floats to integers
df_collide = df_collide.astype(int)
df_collide.dtypes

SEVERITYCODE    int64
LIGHTCOND       int64
ROADCOND        int64
WEATHER         int64
dtype: object

In [15]:
df_collide.head()

,SEVERITYCODE,LIGHTCOND,ROADCOND,WEATHER
0,2,0,1,2
1,1,3,1,3
2,1,0,0,2
3,1,0,0,0
4,2,0,1,3


In [16]:
df_collide['SEVERITYCODE'].value_counts()

1    132285
2     57052
Name: SEVERITYCODE, dtype: int64

In [17]:
#Separate the dataframe into two different ones by 'SEVERITYCODE'
df_majority = df_collide[df_collide['SEVERITYCODE']==1]
df_minority = df_collide[df_collide['SEVERITYCODE']==2]

In [18]:
df_majority.shape

(132285, 4)

In [19]:
df_minority.shape

(57052, 4)

In [20]:
from sklearn.utils import resample

In [21]:
#Downsample the majority and then combine with minority dataframe to balance
df_majority_downsampled = resample(df_majority, 
                                 replace=False,   
                                 n_samples=57052,     
                                 random_state=123) 

In [22]:
df_collide_bal = pd.concat([df_majority_downsampled, df_minority])

In [23]:
df_collide_bal['SEVERITYCODE'].value_counts()

2    57052
1    57052
Name: SEVERITYCODE, dtype: int64

In [22]:
#Define X and Y
X = np.asarray(df_collide_bal[['LIGHTCOND','ROADCOND','WEATHER']])
Y = np.asarray(df_collide_bal['SEVERITYCODE'])

In [23]:
#Normalize the dataset
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-0.63041965, -0.40700352, -0.59684394],
       [-0.63041965, -0.40700352, -0.59684394],
       [-0.63041965, -0.40700352, -0.59684394],
       [-0.63041965, -0.40700352,  0.18897125],
       [ 0.8029545 ,  0.11758452,  0.58187885]])

In [37]:
#Split the data into a testing set and a training set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (85578, 3) (85578,)
Test set: (28526, 3) (28526,)


## Logistic Regression

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LogR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
LogR

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [39]:
yhat = LogR.predict(X_test)
yhat

array([2, 2, 2, ..., 1, 1, 2])

In [40]:
yhat_prob = LogR.predict_proba(X_test)
yhat_prob

array([[0.43863419, 0.56136581],
       [0.43820273, 0.56179727],
       [0.43820273, 0.56179727],
       ...,
       [0.51217947, 0.48782053],
       [0.55851857, 0.44148143],
       [0.43820273, 0.56179727]])

In [41]:
print("CollisionLR Accuracy: ", metrics.accuracy_score(y_test, yhat))

CollisionLR Accuracy:  0.5532146112318587


## Decision Tree

In [42]:
from sklearn.tree import DecisionTreeClassifier
Collision_Tree = DecisionTreeClassifier(criterion="entropy", max_depth = 6)
Collision_Tree
Collision_Tree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [43]:
Treeyhat=Collision_Tree.predict(X_test)
print (Treeyhat [0:5])
print (y_test [0:5])

[2 2 2 2 2]
[2 1 2 2 1]


In [44]:
from sklearn import metrics
print("CollisionTrees's Accuracy: ", metrics.accuracy_score(y_test, Treeyhat))

CollisionTrees's Accuracy:  0.5592441982752576


## K Nearest Neighbors

In [45]:
from sklearn.neighbors import KNeighborsClassifier
k=20

In [46]:
colneigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
colneigh

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=20, p=2,
           weights='uniform')

In [47]:
KNNyhat = colneigh.predict(X_test)
KNNyhat[0:5]

array([2, 2, 2, 2, 2])

In [48]:
print("CollisionKNN Accuracy: ", metrics.accuracy_score(y_test, KNNyhat))

CollisionKNN Accuracy:  0.5509710439598963


## Results

In [49]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [50]:
LRJ = jaccard_similarity_score(y_test, yhat)
LRF1 = f1_score(y_test, yhat, average='weighted')
LRLL = log_loss(y_test, yhat_prob)

print("LOG LogLoss: : %.4f" % LRLL)
print("LOG F1-score: %.4f" % LRF1)
print("LOG Jaccard score: %.4f" % LRJ)

LOG LogLoss: : 0.6691
LOG F1-score: 0.5440
LOG Jaccard score: 0.5532


In [51]:
CTJ = jaccard_similarity_score(y_test, Treeyhat)
CTF1 = f1_score(y_test, Treeyhat, average='weighted')

print("Decision Tree F1-score: %.4f" % CTF1 )
print("Decision Tree Jaccard Score: %.4f" % CTJ)

Decision Tree F1-score: 0.5330
Decision Tree Jaccard Score: 0.5592


In [52]:
KNNJ = jaccard_similarity_score(y_test, KNNyhat)
KNNF1 = f1_score(y_test, KNNyhat, average='weighted')

print("KNN F1-score: %.4f" % KNNF1 )
print("KNN Jaccard Score: %.4f" % KNNJ)

KNN F1-score: 0.5204
KNN Jaccard Score: 0.5510
